# Sumário

- [Tratamento de dados](#Tratamento-de-Dados)
- [Exercícios](#Exercícios)

    - [1. Estes valores estão distribuídos em dados semanais, agrupe eles por mês e calcule as médias de valores de cada combustível por cidade.](#1.)
    - [2. Calcule a média de valor do combustível por estado e região.](#2.)
    - [3. Calcule a variância e a variação absoluta do máximo, mínimo de cada cidade, mês a mês.](#3.)
    - [4. Quais são as 5 cidades que possuem a maior diferença entre o combustível mais barato e o mais caro.](#4.)

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.window import Window

#Criação de uma variável de ambiente, vai instanciando uma sessão Spark
spark = SparkSession.builder.appName("Processa dados massivos").master("local[4]").getOrCreate()

In [2]:
df = spark.read.csv("SEMANAL_MUNICIPIOS-2019.csv",
                    inferSchema=True,
                    header =True,
                    sep=",",
                    encoding = 'utf-8',
                    
)

In [3]:
df.toPandas()

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,30/12/2018,5/1/2019,NORTE,PARA,ABAETETUBA,ETANOL HIDRATADO,1,R$/l,"4,650","0,000","4,650","4,650",-,"0,000",-,-,-,-,-
1,30/12/2018,5/1/2019,NORDESTE,MARANHAO,ACAILANDIA,ETANOL HIDRATADO,2,R$/l,"3,524","0,389","3,249","3,799",-,"0,110",-,-,-,-,-
2,30/12/2018,5/1/2019,SUDESTE,SAO PAULO,ADAMANTINA,ETANOL HIDRATADO,8,R$/l,"2,599","0,105","2,450","2,690","0,243","0,040","2,356","0,078","2,269","2,420","0,033"
3,30/12/2018,5/1/2019,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,"3,031","0,110","2,939","3,199","0,359","0,036","2,672","0,024","2,655","2,689","0,009"
4,30/12/2018,5/1/2019,NORDESTE,BAHIA,ALAGOINHAS,ETANOL HIDRATADO,9,R$/l,"3,220","0,170","2,920","3,320","0,404","0,053","2,816","0,090","2,725","2,904","0,032"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73307,28/7/2019,3/8/2019,SUL,SANTA CATARINA,TUBARAO,GNV,1,R$/m3,"2,949","0,000","2,949","2,949","0,951","0,000","1,998","0,000","1,998","1,998","0,000"
73308,28/7/2019,3/8/2019,SUDESTE,RIO DE JANEIRO,VALENCA,GNV,1,R$/m3,"4,190","0,000","4,190","4,190",-,"0,000",-,-,-,-,-
73309,28/7/2019,3/8/2019,SUDESTE,ESPIRITO SANTO,VILA VELHA,GNV,1,R$/m3,"3,299","0,000","3,299","3,299",-,"0,000",-,-,-,-,-
73310,28/7/2019,3/8/2019,SUDESTE,ESPIRITO SANTO,VITORIA,GNV,2,R$/m3,"3,295","0,006","3,290","3,299",-,"0,002",-,-,-,-,-


## Tratamento de Dados

In [4]:
df.printSchema()

root
 |-- DATA INICIAL: string (nullable = true)
 |-- DATA FINAL: string (nullable = true)
 |-- REGIÃO: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- MUNICÍPIO: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- NÚMERO DE POSTOS PESQUISADOS: integer (nullable = true)
 |-- UNIDADE DE MEDIDA: string (nullable = true)
 |-- PREÇO MÉDIO REVENDA: string (nullable = true)
 |-- DESVIO PADRÃO REVENDA: string (nullable = true)
 |-- PREÇO MÍNIMO REVENDA: string (nullable = true)
 |-- PREÇO MÁXIMO REVENDA: string (nullable = true)
 |-- MARGEM MÉDIA REVENDA: string (nullable = true)
 |-- COEF DE VARIAÇÃO REVENDA: string (nullable = true)
 |-- PREÇO MÉDIO DISTRIBUIÇÃO: string (nullable = true)
 |-- DESVIO PADRÃO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÍNIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÁXIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- COEF DE VARIAÇÃO DISTRIBUIÇÃO: string (nullable = true)



- Convertendo datas para o tipo correto:

In [5]:
# iterar colunas to tipo de data convertendo-as
for col in ["DATA INICIAL","DATA FINAL"]:
    df = df.withColumn(col, sf.to_date(col,"d/M/y"))

- Convertendo colunas com valores decimais para o tipo float:

In [6]:
# source: https://stackoverflow.com/questions/52702608/pyspark-how-to-specify-column-with-comma-as-decimal
for col in df.columns[8:]:
    # replace de virgula por ponto
    df = df.withColumn(col, sf.regexp_replace(col, ',', '.'))
    # convertendo tipo
    df = df.withColumn(col, df[col].cast("float"))

In [7]:
# checando rapidamente se houve alguma perda informação em relação ao dataframe anterior
df.toPandas()

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2018-12-30,2019-01-05,NORTE,PARA,ABAETETUBA,ETANOL HIDRATADO,1,R$/l,4.650,0.000,4.650,4.650,NaN,0.000,NaN,NaN,NaN,NaN,NaN
1,2018-12-30,2019-01-05,NORDESTE,MARANHAO,ACAILANDIA,ETANOL HIDRATADO,2,R$/l,3.524,0.389,3.249,3.799,NaN,0.110,NaN,NaN,NaN,NaN,NaN
2,2018-12-30,2019-01-05,SUDESTE,SAO PAULO,ADAMANTINA,ETANOL HIDRATADO,8,R$/l,2.599,0.105,2.450,2.690,0.243,0.040,2.356,0.078,2.269,2.420,0.033
3,2018-12-30,2019-01-05,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,5,R$/l,3.031,0.110,2.939,3.199,0.359,0.036,2.672,0.024,2.655,2.689,0.009
4,2018-12-30,2019-01-05,NORDESTE,BAHIA,ALAGOINHAS,ETANOL HIDRATADO,9,R$/l,3.220,0.170,2.920,3.320,0.404,0.053,2.816,0.090,2.725,2.904,0.032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73307,2019-07-28,2019-08-03,SUL,SANTA CATARINA,TUBARAO,GNV,1,R$/m3,2.949,0.000,2.949,2.949,0.951,0.000,1.998,0.000,1.998,1.998,0.000
73308,2019-07-28,2019-08-03,SUDESTE,RIO DE JANEIRO,VALENCA,GNV,1,R$/m3,4.190,0.000,4.190,4.190,NaN,0.000,NaN,NaN,NaN,NaN,NaN
73309,2019-07-28,2019-08-03,SUDESTE,ESPIRITO SANTO,VILA VELHA,GNV,1,R$/m3,3.299,0.000,3.299,3.299,NaN,0.000,NaN,NaN,NaN,NaN,NaN
73310,2019-07-28,2019-08-03,SUDESTE,ESPIRITO SANTO,VITORIA,GNV,2,R$/m3,3.295,0.006,3.290,3.299,NaN,0.002,NaN,NaN,NaN,NaN,NaN


In [8]:
# chacando se todos os tipos estão corretos
df.printSchema()

root
 |-- DATA INICIAL: date (nullable = true)
 |-- DATA FINAL: date (nullable = true)
 |-- REGIÃO: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- MUNICÍPIO: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- NÚMERO DE POSTOS PESQUISADOS: integer (nullable = true)
 |-- UNIDADE DE MEDIDA: string (nullable = true)
 |-- PREÇO MÉDIO REVENDA: float (nullable = true)
 |-- DESVIO PADRÃO REVENDA: float (nullable = true)
 |-- PREÇO MÍNIMO REVENDA: float (nullable = true)
 |-- PREÇO MÁXIMO REVENDA: float (nullable = true)
 |-- MARGEM MÉDIA REVENDA: float (nullable = true)
 |-- COEF DE VARIAÇÃO REVENDA: float (nullable = true)
 |-- PREÇO MÉDIO DISTRIBUIÇÃO: float (nullable = true)
 |-- DESVIO PADRÃO DISTRIBUIÇÃO: float (nullable = true)
 |-- PREÇO MÍNIMO DISTRIBUIÇÃO: float (nullable = true)
 |-- PREÇO MÁXIMO DISTRIBUIÇÃO: float (nullable = true)
 |-- COEF DE VARIAÇÃO DISTRIBUIÇÃO: float (nullable = true)



- Confirmar que **produto** e **medida** não têm combinações duplicadas como mesmo produto e duas unidades de medidas diferentes.

In [9]:
df.dropDuplicates(['PRODUTO', 'UNIDADE DE MEDIDA']).select(['PRODUTO', 'UNIDADE DE MEDIDA']).show()

+----------------+-----------------+
|         PRODUTO|UNIDADE DE MEDIDA|
+----------------+-----------------+
|             GNV|            R$/m3|
|             GLP|          R$/13Kg|
|ETANOL HIDRATADO|             R$/l|
|  GASOLINA COMUM|             R$/l|
|     ÓLEO DIESEL|             R$/l|
| ÓLEO DIESEL S10|             R$/l|
+----------------+-----------------+



In [10]:
# Olhar todas as combinações de datas disponíveis
df.dropDuplicates(
                ['DATA INICIAL', 'DATA FINAL']
            ).select(
                ['DATA INICIAL', 'DATA FINAL']
            ).orderBy("DATA INICIAL", ascending=True).show()

+------------+----------+
|DATA INICIAL|DATA FINAL|
+------------+----------+
|  2018-12-30|2019-01-05|
|  2019-01-06|2019-01-12|
|  2019-01-13|2019-01-19|
|  2019-01-20|2019-01-26|
|  2019-01-27|2019-02-02|
|  2019-02-03|2019-02-09|
|  2019-02-10|2019-02-16|
|  2019-02-17|2019-02-23|
|  2019-02-24|2019-03-02|
|  2019-03-03|2019-03-09|
|  2019-03-10|2019-03-16|
|  2019-03-17|2019-03-23|
|  2019-03-24|2019-03-30|
|  2019-03-31|2019-04-06|
|  2019-04-07|2019-04-13|
|  2019-04-14|2019-04-20|
|  2019-04-21|2019-04-27|
|  2019-04-28|2019-05-04|
|  2019-05-05|2019-05-11|
|  2019-05-12|2019-05-18|
+------------+----------+
only showing top 20 rows



df.withColumn('report_date_10', F.date_add(df_dc['report_date'], 10)).show()

## Exercícios

### 1.
**Estes valores estão distribuídos em dados semanais, agrupe eles por mês e calcule as médias de valores de cada combustível por cidade.**

- Fiquei em dúvida se realmente o objetivo seria agrupar só por **mês**, me parece que faria mais sentido por **ano** e por **mês** e assim o fiz.

- Também fiquei em dúvida se utilizava **DATA INICIAL** ou **DATA FINAL**, inicialmente acabei optando por **INICIAL** pois achei que faria mais sentido me referir a cada linha como **semana x inicio** em vez de **semana x fim**. Também pensei em pegar o dia médio de cada semana, mas não me senti tão compelido por essa idéia embora fosse boa.

- Porém depois pensei se eu pegasse o dia médio, eu estaria claramente selecionando no minimo 4 dias da semana como indicador do mês correto, do total de 7 dias, logo seria a melhor lógica e assim o fiz. 

- Outra dúvida também foi qual valor usar, **REVENDA** ou **DISTRIBUIÇÃO**, acabei optando pela coluna **PREÇO MÉDIO** de ambas, separadamente.

In [11]:
# criando coluna data média adicionando 3 dias à data inicial
df = df.withColumn('DATA MÉDIA', sf.date_add(df['DATA INICIAL'], 3))

In [12]:
# criando colunas mês e ano a partir de data média
df = df.withColumn("ANO MÉDIO", sf.year("DATA MÉDIA"))
df = df.withColumn("MES MÉDIO", sf.month("DATA MÉDIA"))

In [13]:
media_combustivel_p_cidade_p_mês = df.groupBy("ANO MÉDIO", "MES MÉDIO", "MUNICÍPIO", "PRODUTO"
    ).agg(
        sf.avg("PREÇO MÉDIO REVENDA").alias("PREÇO MÉDIO REVENDA"),
        sf.avg("PREÇO MÉDIO DISTRIBUIÇÃO").alias("PREÇO MÉDIO DISTRIBUIÇÃO")
    )

In [14]:
media_combustivel_p_cidade_p_mês.orderBy("PREÇO MÉDIO REVENDA", ascending=True).show(50)

+---------+---------+--------------------+----------------+-------------------+------------------------+
|ANO MÉDIO|MES MÉDIO|           MUNICÍPIO|         PRODUTO|PREÇO MÉDIO REVENDA|PREÇO MÉDIO DISTRIBUIÇÃO|
+---------+---------+--------------------+----------------+-------------------+------------------------+
|     2019|        1|                 POA|             GNV| 2.1589999198913574|                    null|
|     2019|        2|                 POA|             GNV| 2.1589999198913574|                    null|
|     2019|        1|     ITAQUAQUECETUBA|             GNV|  2.178999900817871|                    null|
|     2019|        2|          MOGI MIRIM|             GNV| 2.1989998817443848|                    null|
|     2019|        1|          MOGI MIRIM|             GNV| 2.1989998817443848|                    null|
|     2019|        1|           SAO PAULO|             GNV|  2.218400001525879|       1.859000027179718|
|     2019|        1|           GUARULHOS|             

In [15]:
media_combustivel_p_cidade_p_mês.orderBy("PREÇO MÉDIO REVENDA", ascending=False).show(50)

+---------+---------+-------------+-------+-------------------+------------------------+
|ANO MÉDIO|MES MÉDIO|    MUNICÍPIO|PRODUTO|PREÇO MÉDIO REVENDA|PREÇO MÉDIO DISTRIBUIÇÃO|
+---------+---------+-------------+-------+-------------------+------------------------+
|     2019|        1|      SORRISO|    GLP|            108.375|       79.30000305175781|
|     2019|        1|ALTA FLORESTA|    GLP| 105.03339996337891|                    83.6|
|     2019|        1|        SINOP|    GLP| 103.16699981689453|       80.13800048828125|
|     2019|        2|ALTA FLORESTA|    GLP|            103.125|                    86.0|
|     2019|        6|ALTA FLORESTA|    GLP|              103.0|                    86.0|
|     2019|        2|        SINOP|    GLP| 102.70849990844727|                    81.0|
|     2019|        2|      SORRISO|    GLP|          102.34375|       79.30000305175781|
|     2019|        3|        SINOP|    GLP| 101.97900009155273|                    81.0|
|     2019|        5|

### 2.
**Calcule a média de valor do combustível por estado e região.**

In [16]:
media_combustivel_p_estado_regiao = df.groupBy("REGIÃO", "ESTADO", "PRODUTO"
    ).agg(
        sf.avg("PREÇO MÉDIO REVENDA").alias("PREÇO MÉDIO REVENDA"),
        sf.avg("PREÇO MÉDIO DISTRIBUIÇÃO").alias("PREÇO MÉDIO DISTRIBUIÇÃO")
    )

In [17]:
media_combustivel_p_estado_regiao.orderBy("PREÇO MÉDIO REVENDA", ascending=True).show()

+------------+------------------+----------------+-------------------+------------------------+
|      REGIÃO|            ESTADO|         PRODUTO|PREÇO MÉDIO REVENDA|PREÇO MÉDIO DISTRIBUIÇÃO|
+------------+------------------+----------------+-------------------+------------------------+
|CENTRO OESTE|       MATO GROSSO|ETANOL HIDRATADO|   2.72236866335715|      2.3106635891347436|
|     SUDESTE|         SAO PAULO|ETANOL HIDRATADO| 2.7265355418232606|       2.358346940160227|
|    NORDESTE|        PERNAMBUCO|             GNV|  2.831416569650173|      1.9397655986249447|
|CENTRO OESTE|MATO GROSSO DO SUL|             GNV|  2.923177782694499|      2.0758148343474776|
|         SUL|    SANTA CATARINA|             GNV|  2.928991328055302|       2.061190467505228|
|CENTRO OESTE|             GOIAS|ETANOL HIDRATADO| 2.9607571166651976|       2.536651170068933|
|         SUL|            PARANA|ETANOL HIDRATADO|  2.972672971920654|      2.5948425262823873|
|     SUDESTE|         SAO PAULO|       

In [18]:
media_combustivel_p_estado_regiao.orderBy("PREÇO MÉDIO REVENDA", ascending=False).show()

+------------+-------------------+-------+-------------------+------------------------+
|      REGIÃO|             ESTADO|PRODUTO|PREÇO MÉDIO REVENDA|PREÇO MÉDIO DISTRIBUIÇÃO|
+------------+-------------------+-------+-------------------+------------------------+
|CENTRO OESTE|        MATO GROSSO|    GLP|  96.64785689252862|       75.12819952434964|
|       NORTE|            RORAIMA|    GLP|  84.16338717552924|       71.50980451030117|
|       NORTE|               PARA|    GLP|  82.08740120549356|       62.44162699592559|
|       NORTE|           RONDONIA|    GLP|  81.72840873144006|        64.5161720604025|
|       NORTE|          TOCANTINS|    GLP|  79.13297439082976|        56.5605225593813|
|       NORTE|           AMAZONAS|    GLP|  78.96468363115865|       65.01217513545868|
|       NORTE|               ACRE|    GLP|  78.16096779607958|       69.50541920815745|
|       NORTE|              AMAPA|    GLP|  77.51766131001133|       57.66022540676978|
|CENTRO OESTE| MATO GROSSO DO SU

### 3.
**Calcule a variância e a variação absoluta do máximo, mínimo de cada cidade, mês a mês.**

- Confesso que o fato de ter **mês a mês** explícito aqui me deixou confuso sobre minha decisão na questão 1, mas não acho que signifique um porto seguro o suficiente para mudar minha abordagem na primeira questão para agrupar só **por mês** em vez de **por mês e por ano**.

- Também fiquei na dúvida se deveria comparar **máximo com máximo** ou **máximo com mínimo** para cálculo da variância ou variação. Acabei optando pela interpretação que mais fazia sentido para mim dado o enunciado.

- Também considerei utilizar as colunas **COEF DE VARIAÇÃO** e **DESVIO PADRÃO** para chegar na **variância** e **variação**, mas não me pareceu uma idéia muito concreta, embora possível.

- Também optei por adicionar o grão **PRODUTO**

In [19]:
var_df = df.groupBy("MUNICÍPIO", "PRODUTO", "ANO MÉDIO","MES MÉDIO",
        ).agg(
            sf.variance("PREÇO MÍNIMO REVENDA").alias("VARIANCIA PREÇO MÍNIMO REVENDA"),
            sf.variance("PREÇO MÁXIMO REVENDA").alias("VARIANCIA PREÇO MÁXIMO REVENDA"),
            sf.variance("PREÇO MÍNIMO DISTRIBUIÇÃO").alias("VARIANCIA PREÇO MÍNIMO DISTRIBUIÇÃO"),
            sf.variance("PREÇO MÁXIMO DISTRIBUIÇÃO").alias("VARIANCIA PREÇO MÁXIMO DISTRIBUIÇÃO"),
            (sf.max("PREÇO MÍNIMO REVENDA") - sf.min("PREÇO MÍNIMO REVENDA")).alias("VARIAÇÃO PREÇO MÍNIMO REVENDA"),
            (sf.max("PREÇO MÁXIMO REVENDA") - sf.min("PREÇO MÁXIMO REVENDA")).alias("VARIAÇÃO PREÇO MÁXIMO REVENDA"),
            (sf.max("PREÇO MÍNIMO DISTRIBUIÇÃO") - sf.min("PREÇO MÍNIMO DISTRIBUIÇÃO")).alias("VARIAÇÃO PREÇO MÍNIMO DISTRIBUIÇÃO"),
            (sf.max("PREÇO MÁXIMO DISTRIBUIÇÃO") - sf.min("PREÇO MÁXIMO DISTRIBUIÇÃO")).alias("VARIAÇÃO PREÇO MÁXIMO DISTRIBUIÇÃO")
        )

In [20]:
var_df.orderBy(["MUNICÍPIO", "PRODUTO", "ANO MÉDIO","MES MÉDIO"], ascending=True).toPandas().head(20)

,MUNICÍPIO,PRODUTO,ANO MÉDIO,MES MÉDIO,VARIANCIA PREÇO MÍNIMO REVENDA,VARIANCIA PREÇO MÁXIMO REVENDA,VARIANCIA PREÇO MÍNIMO DISTRIBUIÇÃO,VARIANCIA PREÇO MÁXIMO DISTRIBUIÇÃO,VARIAÇÃO PREÇO MÍNIMO REVENDA,VARIAÇÃO PREÇO MÁXIMO REVENDA,VARIAÇÃO PREÇO MÍNIMO DISTRIBUIÇÃO,VARIAÇÃO PREÇO MÁXIMO DISTRIBUIÇÃO
0,ABAETETUBA,ETANOL HIDRATADO,2019,1,0.000000,0.000000,NaN,NaN,0.000,0.00,NaN,NaN
1,ABAETETUBA,ETANOL HIDRATADO,2019,2,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000
2,ABAETETUBA,ETANOL HIDRATADO,2019,3,0.000000,0.000000,0.116162,0.116162,0.000,0.00,0.482,0.482
3,ABAETETUBA,ETANOL HIDRATADO,2019,4,0.003333,0.003333,0.011556,0.011556,0.100,0.10,0.215,0.215
4,ABAETETUBA,ETANOL HIDRATADO,2019,5,0.254300,0.006080,0.000000,0.000000,0.951,0.22,0.000,0.000
5,ABAETETUBA,ETANOL HIDRATADO,2019,6,0.000027,0.000000,NaN,NaN,0.009,0.00,NaN,NaN
6,ABAETETUBA,ETANOL HIDRATADO,2019,7,0.229408,0.000000,NaN,NaN,1.071,0.00,NaN,NaN
7,ABAETETUBA,GASOLINA COMUM,2019,1,0.000000,0.004500,0.000386,0.000386,0.000,0.15,0.039,0.039
8,ABAETETUBA,GASOLINA COMUM,2019,2,0.003333,0.018225,0.000380,0.003106,0.100,0.27,0.039,0.112
9,ABAETETUBA,GASOLINA COMUM,2019,3,0.000000,0.016300,0.005751,0.000674,0.000,0.27,0.174,0.062


### 4.
**Quais são as 5 cidades que possuem a maior diferença entre o combustível mais barato e o mais caro.**

- Decidi comparar tanto preço de **distribuição** quanto **revenda**

In [21]:
dif_df = df.groupby(
        "MUNICÍPIO"
    ).agg(
        (sf.max("PREÇO MÁXIMO DISTRIBUIÇÃO") - sf.min("PREÇO MÍNIMO DISTRIBUIÇÃO")).alias("DIFERENÇA DISTRIBUIÇÃO"),
        (sf.max("PREÇO MÁXIMO REVENDA") - sf.min("PREÇO MÍNIMO REVENDA")).alias("DIFERENÇA REVENDA"),
    )

- Top 5 cidades pela diferença

In [22]:
dif_df.orderBy("DIFERENÇA DISTRIBUIÇÃO",ascending=False).select("MUNICÍPIO").limit(5).show()

+-------------+
|    MUNICÍPIO|
+-------------+
|       CUIABA|
|ALTA FLORESTA|
|VARZEA GRANDE|
|        SINOP|
|      VILHENA|
+-------------+



In [23]:
dif_df.orderBy("DIFERENÇA DISTRIBUIÇÃO",ascending=False).show()

+--------------------+----------------------+-----------------+
|           MUNICÍPIO|DIFERENÇA DISTRIBUIÇÃO|DIFERENÇA REVENDA|
+--------------------+----------------------+-----------------+
|              CUIABA|                88.097|          112.883|
|       ALTA FLORESTA|                83.701|           107.34|
|       VARZEA GRANDE|                83.087|          102.845|
|               SINOP|                 78.91|          107.611|
|             VILHENA|                77.984|            87.56|
|        RONDONOPOLIS|                 77.92|          107.601|
|             SORRISO|                 77.91|           117.46|
|        BOM DESPACHO|                73.862|           79.351|
|              PALMAS|                73.542|            89.86|
|            XINGUARA|                 73.48|           96.511|
|SAO BERNARDO DO C...|                 73.36|           87.741|
|              ARARAS|                72.893|           82.701|
|               BELEM|             72.60

In [24]:
dif_df.orderBy("DIFERENÇA REVENDA",ascending=False).select("MUNICÍPIO").limit(5).show()

+-------------+
|    MUNICÍPIO|
+-------------+
|      SORRISO|
|       CUIABA|
|        SINOP|
| RONDONOPOLIS|
|ALTA FLORESTA|
+-------------+



In [25]:
dif_df.orderBy("DIFERENÇA REVENDA",ascending=False).show()

+--------------------+----------------------+-----------------+
|           MUNICÍPIO|DIFERENÇA DISTRIBUIÇÃO|DIFERENÇA REVENDA|
+--------------------+----------------------+-----------------+
|             SORRISO|                 77.91|           117.46|
|              CUIABA|                88.097|          112.883|
|               SINOP|                 78.91|          107.611|
|        RONDONOPOLIS|                 77.92|          107.601|
|       ALTA FLORESTA|                83.701|           107.34|
|            ALTAMIRA|                66.927|           106.43|
|            ITAITUBA|                  64.9|           104.48|
|       VARZEA GRANDE|                83.087|          102.845|
|            XINGUARA|                 73.48|           96.511|
|            REDENCAO|             71.590004|            96.35|
|             CACERES|                 67.08|            95.73|
|               COXIM|                55.871|           95.721|
|         PARAGOMINAS|                  

- Como um mato grossense nato de sorriso, não fiquei muito feliz de ver minha cidade nessa 1ª posição.

- Decidi agrupar esses dados no mesmo **grão**, com as colunas **PRODUTO**, e **DATA MÉDIA** e ver como ficaria.

In [26]:
dif_grao_df = df.groupby(
        "MUNICÍPIO", "PRODUTO", "DATA MÉDIA"
    ).agg(
        (sf.max("PREÇO MÁXIMO DISTRIBUIÇÃO") - sf.min("PREÇO MÍNIMO DISTRIBUIÇÃO")).alias("DIFERENÇA DISTRIBUIÇÃO"),
        (sf.max("PREÇO MÁXIMO REVENDA") - sf.min("PREÇO MÍNIMO REVENDA")).alias("DIFERENÇA REVENDA"),
    )

In [27]:
dif_grao_df.groupby(
        "MUNICÍPIO"
    ).agg(
        sf.max("DIFERENÇA DISTRIBUIÇÃO").alias("DIFERENÇA DISTRIBUIÇÃO")
    ).orderBy(
        "DIFERENÇA DISTRIBUIÇÃO", ascending=False
).show()

+--------------------+----------------------+
|           MUNICÍPIO|DIFERENÇA DISTRIBUIÇÃO|
+--------------------+----------------------+
|              CUIABA|                 37.64|
|              SANTOS|             31.210003|
|      RIO DE JANEIRO|                 30.25|
|SAO BERNARDO DO C...|                 30.02|
|              ARARAS|                 28.43|
|  SAO CAETANO DO SUL|             27.759998|
|              PALMAS|                  26.2|
|             DIADEMA|             25.910004|
|         SANTA MARIA|                 25.18|
|               BELEM|                  25.0|
|              ILHEUS|                  24.0|
|       CAXIAS DO SUL|                  23.8|
|           SAO PAULO|             23.470001|
|             ERECHIM|             23.130001|
|      NOVA ANDRADINA|                 22.64|
|             LAJEADO|                 22.64|
|          URUGUAIANA|                 22.04|
|       FOZ DO IGUACU|                  21.7|
|            SALVADOR|            

### Obrigado por lerem até aqui e quem sabe futuramente possamos nos conhecer. Abraço